# Introduction


In this assignment, you will practice building and training Convolutional Neural Networks with Pytorch to solve computer vision tasks.  This assignment includes two sections, each involving different tasks:

(1) Image Classification. Predict image-level category labels on two historically notable image datasets: **CIFAR-10** and **MNIST**.

(2) Image Segmentation. Predict pixel-wise classification (semantic segmentation) on synthetic input images formed by superimposing MNIST images on top of CIFAR images.

You will design your own models in each section and build the entire training/testing pipeline with PyTorch.
PyTorch provides optimized implementations of the building blocks and additional utilities, both of which will be necessary for experiments on real datasets. It is highly recommended to read the official [documentation](https://pytorch.org/docs/stable/index.html) and [examples](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) before starting your implementation. There are some APIs that you'll find useful:
[Layers](http://pytorch.org/docs/stable/nn.html),
[Activations](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity),
[Loss functions](http://pytorch.org/docs/stable/nn.html#loss-functions),
[Optimizers](http://pytorch.org/docs/stable/optim.html)

It is highly recommended to use Google Colab and run the notebook on a GPU node. Check https://colab.research.google.com/ and look for tutorials online. To use a GPU go to Runtime -> Change runtime type and select GPU.


# (2) Image Segmentation
The task consists of performing pixel-wise classification on a synthetic dataset
of 32x32 RGB images. Each image was generated by placing a MNIST sample (a grayscale
image of a digit between 0 and 9) on top of a CIFAR-10 sample (a RGB image drawn from
one of 10 possible classes). Each image has an accompanying target tensor of size 32x32,
where in each pixel location (i,j) it contains the ground-truth label of the MNIST digit
(ranging from 0 to 9) or of the CIFAR-10 image (ranging from 10 to 19), depending on whether
the (i,j) pixel in the original image belongs to the superposed MNIST image or not. The
metric of interest here is pixel-wise accuracy, which is the fraction of pixels in each image
for which your model predicted the correct class (out of a total of 20 classes, as described
above).

Note that there are many ways to frame the above task. For example, your CNN can directly
output a 20x32x32 tensor for each input image, representing a distribution over the possible
20 classes for each of the 32x32 pixels. However, note that the problem has a lot of additional
structure: for example, each 32x32 target tensor only has two distinct numbers in it, the label
of the MNIST digit and the label of the CIFAR-10 background image -- accounting for such
structure will make training faster and likely improve your model's final performance. Your
model should be able to achieve around 70% accuracy on the test set when trained for 100 epochs.

To finish this section step by step, you need to:

* Prepare data by building a dataset and data loader. (already provided below)

* Implement training code (6 points) & testing code (6 points), including saving and loading of models.

* Construct a model (12 points) and choose an optimizer (3 points).

* Describe what you did, any additional features you implemented, and/or any graphs you made in training and evaluating your network. Report final test accuracy @100 epochs in a writeup: hw3.pdf (3 points)

In [ ]:
import numpy as np
import os
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import sampler
import torchvision
import torchvision.transforms as T

#NOTE: manually added data loading utils for collab environment fix
#from utils import SegDataset $
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets

class SegDataset(Dataset):
    def __init__(self, root_path='./data', train=True, transform=None):
        super(SegDataset, self).__init__()
        self.transform = transform

        try:
            if train:
                self.data = torch.load(os.path.join(root_path, "train_data.pth")).float() / 255.
                self.target = torch.load(os.path.join(root_path, "train_target.pth")).long()
            else:
                self.data = torch.load(os.path.join(root_path, "test_data.pth")).float() / 255.
                self.target = torch.load(os.path.join(root_path, "test_target.pth")).long()
        except:
            print("Error when loading data: {train, test}_{data, target}.pth should be in data folder.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data[idx]
        target = self.target[idx]
        if self.transform is not None:
            data = self.transform(data)
        return data, target


## Data Preparation:

Setup a Dataset for training and testing.

Datasets load single training examples one a time, so we practically wrap each Dataset in a DataLoader, which loads a data batch in parallel.

In [ ]:
seg_train = SegDataset(root_path='./data', train=True, transform=None) # in my testing in collab I modified ./data to ., as I was loading .pth data files manually into collab
loader_train = DataLoader(seg_train, batch_size=64, shuffle=True)
seg_test = SegDataset(root_path='./data', train=False, transform=None)
loader_test = DataLoader(seg_test, batch_size=64, shuffle=False)

## Design/choose your own model structure (12 points) and optimizer (3 points).
You might want to adjust following configurations for better performance:

(1) Network architecture:
- You can borrow some ideas from existing convnets design, e.g., [ResNet](https://arxiv.org/abs/1512.03385) where
the input from the previous layer is added to the output, or [UNet](https://arxiv.org/pdf/1505.04597.pdf) where you can stack intermediate features from previous layers.
- Note: Do not **directly copy** an existing network design.

(2) Architecture hyperparameters:
- Filter size, number of filters, and number of layers (depth). Make careful choices to tradeoff computational efficiency and accuracy.
- Pooling vs. Strided Convolution
- Batch normalization
- Choice of non-linear activation

(3) Choice of optimizer (e.g., SGD, Adam, Adagrad, RMSprop) and associated hyperparameters (e.g., learning rate, momentum).


In [ ]:
# resblock architecture with striding
# same ResBlock architecture from CIFAR10 implimentation
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResBlock, self).__init__()
        # convolution / batch norm blocks
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = nn.Sequential()
        # strided downsampling
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.downsample(identity)
        out = self.relu(out)
        return out

class PixelWiseClassifier(nn.Module):
    def __init__(self):
        super(PixelWiseClassifier, self).__init__()
        # convolution, batchnorm, resblock layers
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.resblock1 = ResBlock(64, 64)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.resblock2 = ResBlock(128, 128)

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.resblock3 = ResBlock(256, 256)

        # simple convolution, batch norm layers to add model strength
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)

        self.conv5 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)

        self.conv6 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(512)

        # another convolution, batchnorm, resblock layers
        self.resblock4 = ResBlock(512, 512)
        self.conv7 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn7 = nn.BatchNorm2d(512)

        # another convolution, batch norm layers
        self.conv8 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn8 = nn.BatchNorm2d(512)

        self.resblock5 = ResBlock(512, 512)
        self.conv9 = nn.Conv2d(512, 20, kernel_size=1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.resblock1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.resblock2(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.resblock3(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.resblock4(x)
        x = F.relu(self.bn7(self.conv7(x)))
        x = F.relu(self.bn8(self.conv8(x)))
        x = self.resblock5(x)
        x = self.conv9(x)

        return x



## Training (6 points)

Train a model on the given dataset using the PyTorch Module API.

Inputs:
- loader_train: The loader from which train samples will be drawn from.
- loader_test: The loader from which test samples will be drawn from
- model: A PyTorch Module giving the model to train.
- optimizer: An Optimizer object we will use to train the model
- epochs: (Optional) A Python integer giving the number of epochs to train for

Returns: Nothing, but prints model accuracies during training.

In [ ]:
def train(loader_train, loader_test, model, optimizer, epochs=100):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    # standard cross entropy loss function
    criterion = F.cross_entropy

    for e in range(epochs):
        model.train()
        for t, (x, y) in enumerate(loader_train):
            # (1) move data to GPU
            x, y = x.to(device), y.to(device)

            # (2) forward and get loss
            output = model(x)
            loss = criterion(output, y)

            # (3) zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # (4) the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # (5) update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            # Only print out every 25 steps
            if t % 25 == 0:
                print('Epoch %d, Iteration %d, loss = %.4f' % (e, t, loss.item()))

        test(loader_test, model)

        # save model for partial model training functionality
        torch.save(model.state_dict(), "model.pth")

## Testing (6 points)
Test a model using the PyTorch Module API.

Inputs:
- loader: The loader from which test samples will be drawn from.
- model: A PyTorch Module giving the model to test.

Returns: Nothing, but prints model accuracies during training.

In [ ]:
def test(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval() # set model to evaluation mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    with torch.no_grad():
        for x, y in loader:
            # (1) move to GPU
            x, y = x.to(device), y.to(device)

            # (2) forward and calculate scores and predictions
            output = model(x)
            _, predicted = torch.max(output.data, 1)

            # (3) accumulate num_correct and num_samples
            num_samples += y.size(0) * y.size(1) * y.size(2)
            num_correct += (predicted == y).sum().item()


    acc = num_correct / num_samples
    print('Eval %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [ ]:
# load model util for partial model training when model is saved
def load_model(model, load_path):
    model.load_state_dict(torch.load(load_path))
    print(f"Model loaded from {load_path}")
    return model

Describe your design details in the writeup hw3.pdf. (3 points)

Finish your model and optimizer below.

# Data Segmentation Implementation

In [ ]:
lr = 0.01 # standard learning rate, momentum, decay rate
momentum = 0.9
weight_decay = 0.01
model = PixelWiseClassifier()
optimizer = optim.SGD(model.parameters(), lr, momentum, weight_decay)
train(loader_train, loader_test, model, optimizer, epochs=100)

# loaded_model = load_model(PixelWiseClassifier(), "model.pth") #example model loading for partial training